# Исследование надежности заемщиковв

## Обзор данных

In [280]:
#Импортируем pandas, считываем данные из data.csv, сохраняем в переменную data
import pandas as pd

try:
    data = pd.read_csv('data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [282]:
#Смотрим первые 20 строк датафрейма
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [284]:
#Смотрим основную информацию
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


*В датафрейме есть пропущенные значения в 'days_employed' и 'total_income'*

In [287]:
#Выведем их
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

## Предобработка данных

In [289]:
#Смотрим медианное значение для среднего дохода относительно типа занятости
data.groupby('income_type')['total_income'].agg('median')

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

In [292]:
#Заменяем пропущенные значения в 'total_income' медианным значением относительно типа занятости
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

In [294]:
#Мы видим отрицательные значения в столбце трудового стажа, такого быть не может, заменяем их 
data['days_employed'] = data['days_employed'].abs()

In [296]:
#Смотрим медианное значение для дней трудовохо стажа относительно типа занятости
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть.

In [299]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [301]:
#Проверяем на отсутсвие пропусков
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [303]:
#Проверяем уникальные значения для количества детей
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

Видим, что аномалиями здесь являются количественные показтели -1 и 20, избавимся от них

In [306]:
#Удаляем значения
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [308]:
#Проверяем уникальные значения после обработки
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

In [310]:
#Заменяем вещественный тип данных в 'total_income' на int
data['total_income'] = data['total_income'].astype(int)

In [312]:
#обрабатываем неявные дубликаты в 'education'
data['education'] = data['education'].str.lower()

In [314]:
#Ищем дубликаты и удаляем их
data.duplicated().sum()

71

In [318]:
data = data.drop_duplicates()

Данные обработаны, исправлены пропуски 

## Категоризация данных

In [325]:
#Создадим категории данных относительно уровня дохода клиента, по ТЗ заказчика
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [327]:
#Создадим столбец с категориями 
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [331]:
#Выведем перечень уникальных значений для целей 
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [333]:
#Создадим категории данных относительно целей клиента
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [401]:
#Создадим столбец с категориями 
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследование данных и анализ

### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [405]:
debt_children = data.pivot_table(index='children',
                 values='debt',
                 aggfunc=['count', 'sum', 'mean']).reset_index()

debt_children.columns = ['Кол-во детей', 'Размер сегмента', 'Кол-во должников', 'Доля должников']

debt_children.sort_values(by='Доля должников').style.format({'Доля должников': '{:.1%}'})

,Кол-во детей,Размер сегмента,Кол-во должников,Доля должников
5,5,9,0,0.0%
0,0,14091,1063,7.5%
3,3,330,27,8.2%
1,1,4808,444,9.2%
2,2,2052,194,9.5%
4,4,41,4,9.8%


**Вывод:** 

Мы определили должников в группы по количеству детей:
1. Процент должников без детей 7.5%
2. Процент должников с одним ребенком 9.2%
3. Процент должников с двумя детьми 9.4%
4. Процент должников с тремя детьми 8.1%
5. Процент должников с четырьмя детьми 9.7%
6. Должников с пятью детьми нет

*Клиентов, у которых пять детей и есть долг, не найдено, их также менее одного процента, в выборке учитывать их не будем, по той же причине не будем указывать группы с тремя и четырьмя детьми.*

* Группа клиентов без детей самая большая, при этом они имеет самый низкий процент невозврата долгов - 7.5%

* Группа с одним ребенком явялется второй по величине и имеет покатель в - 9.2% 

* Группа с двумя детьми находится на третьем месте по проценту должников - 9.4%

*Отвечая на вопрос задания, мы выели статистику должников из трех самых круппных групп по количеству детей. Резлуьтатом стала зависимость роста процента должников относительно количества детей.*

###  Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [409]:
debt_family = data.pivot_table(index='family_status',
                 values='debt',
                 aggfunc=['count', 'sum', 'mean']).reset_index()

debt_family.columns = ['Кол-во детей', 'Размер сегмента', 'Кол-во должников', 'Доля должников']

debt_family.sort_values(by='Доля должников').style.format({'Доля должников': '{:.1%}'})

,Кол-во детей,Размер сегмента,Кол-во должников,Доля должников
2,вдовец / вдова,951,63,6.6%
1,в разводе,1189,84,7.1%
4,женат / замужем,12261,927,7.6%
3,гражданский брак,4134,385,9.3%
0,Не женат / не замужем,2796,273,9.8%


**Вывод:** 

1. Процент должников в браке: 7.5%
2. Процент должников в гражданском браке: 9.3%
3. Процент холостых должников: 9.7%
4. Процент должников в разводе: 7%
5. Процент должников потерявших супруга: 6,6%


* Исходя из анализа мы видим, что клиенты вдовцы и находящиеся в разводе являются наименее предрасположенными к задолжности. 

* Холостые клиенты являются самыми частыми должниками, клиенты в гражданском браке находятся в среднем диапазоне просрочек.

* Должники находящиеся в браке составляют самую большую группу клиентов относительно семейного статуса, при этом их показатель задолжности меньше среднего. 

* Показательно, что клиенты, которые состоят или состояли в официальном браке являются менее расположенной к просрочкам группой, что вероятнее всего связано с наколпением общего капитала и уменеем планировать расходы, также в этих группах скорее всего находятся более взрослые клиенты.

*Отвечая на вопрос из задния, исходя из анализа данных холостые чаще просрачивают платежи, чем семейные клиенты, а люди в гражданском браке чаще, чем в официальном, вдовы/цы и разведенные имеют меньше всего процента просрочек. Данных досаточно, чтобы заявить, что семейное положение влияет на возврат долга.*

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [413]:
debt_family = data.pivot_table(index='total_income_category',
                 values='debt',
                 aggfunc=['count', 'sum', 'mean']).reset_index()

debt_family.columns = ['Кол-во детей', 'Размер сегмента', 'Кол-во должников', 'Доля должников']

debt_family.sort_values(by='Доля должников').style.format({'Доля должников': '{:.1%}'})

,Кол-во детей,Размер сегмента,Кол-во должников,Доля должников
3,D,349,21,6.0%
1,B,5014,354,7.1%
0,A,25,2,8.0%
2,C,15921,1353,8.5%
4,E,22,2,9.1%


**Вывод:** 

1. Процент должников категории А: 8%
2. Процент должников категории В: 7.1%
3. Процент холостых категории С: 8.5%
4. Процент должников категории D: 6%
5. Процент должников категории Е: 9.1%

* Исходя из полученных данных категории A и E можно не учитывать в сравнительном анализе, потому что в каждой из групп менее 1% клиентов, также не будем учитывать группу  D, в групппе всего 1.5% клиентов.
* Категория C является наибольшей и статистически именно вней самый высокий процент неплательщиков - 8%
* Категория В вторая по величине и вторая по количеству неплательщиков - 7.1%


*Отвечая на вопрос из задния, исходя из показателей очевидно, что в категории B процент просрочек ниже, чем в категории C, соответсвенно можно заявлять, что количество доли просрочек уменьшается с ростом дохода.*

**Рекомендации:** 

*Если создать более равные группы по категория доходов клиентов, мы сможем получить более точные данные. Это мы можем сделать с помощью метода qcut() из библиотеки pandas:*

In [417]:
#Выводим на экран 5 новых категорий
pd.qcut(data['total_income'], q=5).value_counts()

total_income
(20666.999, 98514.0]     4267
(98514.0, 132113.0]      4266
(132113.0, 161380.0]     4266
(161380.0, 214604.0]     4266
(214604.0, 2265604.0]    4266
Name: count, dtype: int64

In [419]:
#Делим на категории
category_new = ['E', 'D', 'C', 'B', 'A']

data['total_income_new_category'] = pd.qcut(data['total_income'], q=5, labels=category_new)
data['total_income_new_category'].value_counts()

total_income_new_category
E    4267
D    4266
C    4266
B    4266
A    4266
Name: count, dtype: int64

In [421]:
debt_family = data.pivot_table(index='total_income_new_category',
                 values='debt',
                 aggfunc=['count', 'sum', 'mean']).reset_index()

debt_family.columns = ['Кол-во детей', 'Размер сегмента', 'Кол-во должников', 'Доля должников']

debt_family.sort_values(by='Доля должников').style.format({'Доля должников': '{:.1%}'})

C:\Users\ЯУСТАЛ\AppData\Local\Temp\ipykernel_3024\2811893797.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  debt_family = data.pivot_table(index='total_income_new_category',
C:\Users\ЯУСТАЛ\AppData\Local\Temp\ipykernel_3024\2811893797.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  debt_family = data.pivot_table(index='total_income_new_category',
C:\Users\ЯУСТАЛ\AppData\Local\Temp\ipykernel_3024\2811893797.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  debt_family = data.pivot_table(index=

,Кол-во детей,Размер сегмента,Кол-во должников,Доля должников
4,A,4266,299,7.0%
0,E,4267,344,8.1%
1,D,4266,358,8.4%
3,B,4266,358,8.4%
2,C,4266,373,8.7%


**Вывод:** 

1. Процент должников категории А: 7%
2. Процент должников категории В: 8.4%
3. Процент холостых категории С: 8.7%
4. Процент должников категории D: 8.4%
5. Процент должников категории Е: 8.1%

*Разделив группы клиентов на более равные категории, мы смогли подготовить обновленные данные:*

1. Самые богатые клиент допускают меньше всего просрочек
2. Клиенты из 'Cреднего класса' оставили за собой самый большой процент неплательщиков
3. Количество просрочек от категории А до С расет, а от С до Е снижается

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [425]:
debt_family = data.pivot_table(index='purpose_category',
                 values='debt',
                 aggfunc=['count', 'sum', 'mean']).reset_index()

debt_family.columns = ['Кол-во детей', 'Размер сегмента', 'Кол-во должников', 'Доля должников']

debt_family.sort_values(by='Доля должников').style.format({'Доля должников': '{:.1%}'})

,Кол-во детей,Размер сегмента,Кол-во должников,Доля должников
1,операции с недвижимостью,10751,780,7.3%
3,проведение свадьбы,2313,183,7.9%
2,получение образования,3988,369,9.3%
0,операции с автомобилем,4279,400,9.3%


**Вывод:** 

1. Процент должников с целью недвижимость: 7.2%
2. Процент должников с целью авто: 9.3%
3. Процент холостых с целью образование: 9.3%
4. Процент должников с целью свадьба: 7.9%

* Исходя из данных наибольшая группа клиентов находится среди получателей кредита на квартиру, данная группа имеет наименьший процент должников. Вероятно это связано с тем, что при ипотеке имущество находится в залоге у банка и терять клиентам его не хочется


* Должников по кредиту на образование и авто примерно равное количество, также и равен процент невозврата кредита на данные цели - 9.3%. 

1. Причиной невыплат кредита за учебу может стать финансовое положение студентов, которые работают на низкооплачиваемой или нерегулярной работе, в связи с чем часто допускают проссрочи, а также выпускников, которые только выпустились и находятся в поиске новой работы или только на нее устроились. 
2. Среди автолюбителей дополнительно стоило бы оценить возраст автомобиля и его категорию, возможно, машины большинства просрочников слишком старые и они тратят все деньги на их починку.
* Количество клиентов, которые брали кредит на свадьбу меньше всего, они находятся в середине по невыплатам, возможно это связано с тем, что свадьба у клиента не окупилась и платить за кредит им теперь нечем

## Общий вывод.

Для анализа мы получили таблицу заемщиков банка. Проверили ее на пустые значения и дубликаты, обработали аномальные значения, и превели значения к читаемому виду, сделали категоризацию.

Далее мы проанализировали различные группы клиентов банка на предмет неоплаты долга, выявили отдельные зависимости в этих группах.

Мы узнали, что самые большие группы должников в своих категориях:
* Клиенты с четырьмя детьми
* Холостые клиенты
* Клиенты c доходом от 200 001 до 1 000 000
* Клиенты которые взяли кредит на автомобиль и образование

Также мы пришли к выводу, что:

* *Процент должников растет в соответствии с ростом количества детей*

* *Семейное положение непосредственно влияет на возврат долга*

* Самые богатые клиенты допускают меньше всего просрочек

* *Доход  влияет на процент просрочек, самый большой процент просрочек у "среднего класса"* 

* *Клиенты с кредитом на недвижемость допускают меньше всего просрочек*

Мы смогли определить портреты хорошоего и плохого клиента:

* Хороший клиент: Должен состоять или должен был состоять в официальном браке, не имеет детей, обладает высоким доходом, планирующий приобрести недвижимость 

* Плохой клиент: Не состоит в отношениях, имеет несколько детей, имеет средний доход и хочет приобрести автомобиль

Рекомендации по сбору данных для улучшения скоринга:

* Стоит пересмотреть некоторые группы клиентов, например, некоторые отдельные категории людей с детьми объединить, создать категорию 'многодетные' и объединить в ней всех клиентов у кого более одного или двух детей. 
* Также можно пересмотреть категории распределния по доходам, пример того, как это можно сделать находится в отчете.
* Еще можно создать группы не по отдельным параметрам, а по их набору, например: 'Семейные, с детьми и средним доходом', 'Холостые, с детьми и с высоким доходом'. 
* Стоит учитывать крединтную историю клиента из других банков